In [ ]:
pip install datapackage

In [ ]:
import datapackage
import pandas as pd

data_url = 'https://datahub.io/machine-learning/diabetes/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        data = pd.read_csv(resource.descriptor['path'])
        print (data)

In [ ]:
title="class"
new_data=(data.iloc[:,:]) ##Seleção da todas as linhas e colunas
df1=new_data[title]#pd.get_dummies(df[title],drop_first=False)

In [ ]:
new_data

In [ ]:
def categoriza(s):
    if s == 'tested_negative':
       return 0
    else:
       return 1

new_data['class'] = df1.apply(categoriza)

In [ ]:
new_data

In [ ]:
#x=new_data.iloc[:,:-1].values
y=new_data['class'].values

In [ ]:
features=['preg','plas','insu','mass','pedi','age']
x=new_data[features]

In [ ]:
x

In [ ]:
y

In [ ]:
new_data.corr()

In [ ]:
import seaborn as sns

plt.figure(figsize=(30, 20))
sns.heatmap(new_data.corr(),annot = True, fmt = '.2f')
plt.title('Correlação entre variáveis e os movimentos de marcha')
plt.savefig('Correlação entre variáveis e os movimentos de marcha')
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler


## Matriz de covariância ##
#normalização 
sc = StandardScaler()
X= sc.fit_transform(x)
X

In [ ]:
y

In [ ]:
import numpy as np

cov = np.cov(X.T) 
eigenvalues, eigenvectors = np.linalg.eig(cov)



In [ ]:
import matplotlib.pyplot as plt

tot_soma = sum(eigenvalues)
var_exp = [(i / tot_soma) for i in sorted(eigenvalues, reverse=True)] #do maior para o menor valor de autovalores
cum_var_exp = np.cumsum(var_exp)


In [ ]:
X.shape[1]

In [ ]:

plt.bar( range(1,X.shape[1]+1), var_exp, alpha=0.5, color = 'r')

plt.ylabel('Variância em %')
plt.xlabel('Componentes Principais')

plt.show()

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss


nr = NearMiss() #Undersampling
smt = SMOTE() #Oversampling

X, y = smt.fit_sample(X, y)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.5)

In [ ]:
#importing standard neural network libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score # for evaluation
from sklearn.metrics import confusion_matrix,classification_report

classifiers = [
    DecisionTreeClassifier(),
    RandomForestClassifier(max_depth=5, random_state=50),
    KNeighborsClassifier(2),
    SVC(kernel = 'poly', C = 2),
    AdaBoostClassifier(n_estimators=50),
    MLPClassifier(hidden_layer_sizes=(100,1000,100),random_state=1, max_iter=300)    
]

names = []
scores = []
y_Pred=[]
for clf in classifiers:
    clf = clf.fit(X_train, y_train)
    ypred=clf.predict(X_val)
    y_Pred.append(clf.predict(X_val))
    
    names.append(clf.__class__.__name__)
    scores.append(accuracy_score(ypred, y_val))

score_df = pd.DataFrame({'Model': names,'Score': scores})
score_df

In [ ]:
ax = score_df.plot.bar()
ax.set_xticklabels(score_df['Model'], rotation=45, fontsize=10)
plt.savefig('comparacao_entre_os_modelos_scores.png')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from keras.optimizers import RMSprop

model = Sequential([
    Dense(100, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l2(0.00175)),
    Dropout(0.5),
    Dense(1000, kernel_regularizer=regularizers.l2(0.0015),activation='relu'),
    Dropout(0.5),
    Dense(5000, kernel_regularizer=regularizers.l2(0.0015),activation='relu'),
    Dropout(0.5),
    Dense(1000, kernel_regularizer=regularizers.l2(0.0015),activation='relu'),
    Dropout(0.5),
    Dense(100, kernel_regularizer=regularizers.l2(0.0015),activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()


In [ ]:
model.compile(optimizer='adam',
               loss=tf.keras.losses.binary_crossentropy,#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['mse','accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [ ]:
class_weight = {
    0: 1.0,#4.87,
    1: 1.0,#3.639,
    2: 1.0,#5.726,
    3: 1.0,#3.7615,
    4: 1.0,#4.85946,
    5: 1.0,
}

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=0.001, patience=25)

historico = model.fit(X_train, y_train,class_weight=class_weight,shuffle=True,
          batch_size=16, epochs=100,
          validation_data=(X_val, y_val),callbacks=[es])

In [ ]:
model.evaluate(X_val,y_val)

In [ ]:
model.evaluate(X_test,y_test)


In [ ]:
import matplotlib.pyplot as plot


In [ ]:
plot.plot(historico.history['loss'])
plot.plot(historico.history['val_loss'])
plot.title('Model loss')
plot.ylabel('Loss')
plot.xlabel('Epoch')
plot.legend(['Train', 'Val'], loc='upper right')
plt.savefig("Perdas na detecção do diabetes.png")
plot.show()

In [ ]:
plt.plot(historico.history['accuracy'])
plt.plot(historico.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.savefig("Acurácia na detecção do diabetes.png")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict,StratifiedKFold
#from yellowbrick.classifier import ConfusionMatrix
import numpy as np


y_pred = model.predict(X_test)  #cross_val_predict(model, X_train, Y_train, cv = cv)
#print(y_pred)
y_pred=(np.around(y_pred))

In [ ]:

from sklearn.metrics import confusion_matrix,classification_report

confusion_matrix(y_test,y_pred)

In [ ]:

import itertools
import numpy as np
import matplotlib.pyplot as plt


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues, figsize=(24,24)):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() /2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()



In [ ]:

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

plt.figure(figsize=(15, 10))
plot_confusion_matrix(cnf_matrix, classes=['With diabetes','without diabetes'], normalize=True,
                      title='Normalized confusion matrix')
plt.savefig("Matrix Confusao Normalizada.png")
plt.show()

In [ ]:
from sklearn.metrics import roc_curve

print(classification_report(y_test,y_pred,digits=5))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import roc_auc_score
#model = RandomForestClassifier(n_estimators=10,max_depth=6, max_features='auto', max_leaf_nodes=None,random_state=100)
model = RandomForestClassifier(max_depth=5, random_state=100,n_estimators=1000)
model.fit(X_train,y_train)

In [ ]:

score = model.estimators_[0].score(X_val, y_val)
print(score)

In [ ]:
score = model.estimators_[0].score(X_test, y_test)
print(score)

In [ ]:
fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (60,15), dpi = 100)
tree.plot_tree(model.estimators_[0],filled=True, rounded=True) 
fig.savefig('randomforest.png')

In [ ]:
y_pred = model.predict(X_test)  #cross_val_predict(model, X_train, Y_train, cv = cv)
#print(y_pred)
y_pred=(np.around(y_pred))

In [ ]:

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

plt.figure(figsize=(15, 10))
plot_confusion_matrix(cnf_matrix, classes=['With diabetes','without diabetes'], normalize=True,
                      title='Normalized confusion matrix')
plt.savefig("Matrix Confusao Normalizada.png")
plt.show()

In [ ]:
print(classification_report(y_test,y_pred,digits=5))